# Example usage

This notebook aims to provide sample usage of the `classifiers` python package.

First thing is to import the needed objects and modules for these examples.


In [4]:
%load_ext autoreload
%autoreload 2

import tempfile

import torch

from classifiers import (
    AA_TOKENIZER,
    AACnnClassifier,
    AttentionClassifier,
    Data,
    DenseMsaClassifier,
    DenseSiteClassifier,
    FastaSource,
    LogisticRegressionClassifier,
    Pipeline,
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We then determine if the code will run on GPU or CPU.


In [5]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

## Single classifier

A first way to use the package is to run a single classifier from Python code in a script or notebook.

To do that, the first step is to define the sources of both empirical and simulated data. The `FastaSource` class allows to creata a data source from a folder of Fasta files.


In [7]:
real_source = FastaSource("../data/sample_lg_s256_gc/empirical/")
simulated_source = FastaSource("../data/sample_lg_s256_gc/simulated/")

We then define a `Data` object which will contain the base data used by all the classifiers. This is done by passing to the `Data` constructor our two data sources, and the tokenizer to be used (here it is `AA_TOKENIZER` as our dataset contains amino acids sequences).

The `Data` constructor runs common data preprocessing steps:

-   string sequences are tokenized into sequences of integers
-   sequences with ambiguous tokens are removed from alignments
-   labels are created for both datasets: alignments from the empirical dataset get the label 0, whereas simulated alignments get the label 1


In [8]:
data = Data(source_real=real_source, source_simulated=simulated_source, tokenizer=AA_TOKENIZER)

2024-10-07 09:47:12   Tokenizing real alignments...
2024-10-07 09:47:13   Removing sequences with ambiguous tokens...
2024-10-07 09:47:13   127/8988 sequences have been removed from 84/488 alignements due to ambiguous sites.
2024-10-07 09:47:13   Tokenizing simulated alignments...
2024-10-07 09:47:15   Removing sequences with ambiguous tokens...
2024-10-07 09:47:15   0/8713 sequences have been removed from 0/485 alignements due to ambiguous sites.
2024-10-07 09:47:15   Checking alignments...
2024-10-07 09:47:15   485 renamed keys in simulated aligns.
2024-10-07 09:47:15   Creating labels...
2024-10-07 09:47:15   Merging aligns and labels...
2024-10-07 09:47:15   Total number of alignments: 973


### Logistic regression


To run a logistic regression classifier, you must create a `LogisticRegressionClassifier` instance by passing it the base data object, the output path for the results, and some other arguments such as `scale_features`.

The logistic regression classifier works on MSA composition data (overall proportion of amino acids for each alignment).


In [9]:
classif = LogisticRegressionClassifier(
    data=data,
    cv=5,
    shuffle_data=False,
    scale_features=True,
    out_path=tempfile.TemporaryDirectory().name,
)

2024-10-07 09:48:08   --- Preprocessing data ---


Once the classifier instance has been created, its `train()` method can be used to run the classifier. The results of the training is displayed in the standard output and saved in the `out_path` folder.


In [10]:
clf = classif.train()

2024-10-07 09:48:10   --- Start training ---
2024-10-07 09:48:10   Number of cross validation folds: 5
2024-10-07 09:48:10   Shuffle data: False
2024-10-07 09:48:10   Scale features: True
2024-10-07 09:48:10   --- Training ended ---
2024-10-07 09:48:10   Training time: 0:00:00
2024-10-07 09:48:10   Fold F1 scores: ('0.9596', '0.9949', '0.9789', '0.9898', '0.9898')


### Dense network on MSA composition

This classifier is composed of a simple dense neural network which takes the MSA composition data as input.

To run this classifier an instance of `DenseMsaClassifier` must be created with our base data, `out_path`, device and other optional arguments.


In [11]:
dense_msa_classif = DenseMsaClassifier(
    data,
    split_proportion=0.9,
    batch_size=64,
    max_epochs=5,
    device=DEVICE,
    out_path=tempfile.TemporaryDirectory().name,
)

/home/julien/projets/simulations_realistes/classifiers/.venv/lib/python3.12/site-packages/ignite/handlers/tqdm_logger.py:127: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


You can then run the classifier with its `train()` method.


In [12]:
dense_msa_classif.train()

--- Hyperparameters ---
model = DenseMsaNet(
  (dense_layer1): Sequential(
    (0): Linear(in_features=22, out_features=100, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
  )
  (dense_layer2): Linear(in_features=100, out_features=1, bias=True)
)
split_proportion = 0.9
batch_pad_sequences = False
learning_rate = 0.01
batch_size = 64
max_epochs = 5
early_stopping_patience = 5
--- Creating loaders ---
--- Training start ---
Start training using cpu device.
Epoch   Train loss   Valid loss    F1-score        Lr   Best
------------------------------------------------------------


[1/14]   7%|7          [00:00<?]

    1        0.695        0.693       0.000   0.00000      *


[1/14]   7%|7          [00:00<?]

    2        0.689        0.690       0.000   0.00250      *


...

    3        0.679        0.677       0.595   0.00500      *


[1/14]   7%|7          [00:00<?]

    4        0.657        0.657       0.713   0.00750      *


[1/14]   7%|7          [00:00<?]

    5        0.607        0.603       0.622   0.01000      *
--- Training ended ---
Training time: 0:00:01
Best epoch: 5
Best valid loss: 0.603
Best valid accuracy (macro): 0.711
Best F1 score: 0.622


### Dense network on site composition

This is the same type of classifier than the previous one, except that it runs on a per-site composition dataset instead of an overall MSA composition dataset.

To use the classifier you have to create an instance of `DenseSiteClassifier` with its base data, device and `out_path`.


In [13]:
dense_site_classif = DenseSiteClassifier(
    data,
    split_proportion=0.9,
    batch_size=64,
    max_epochs=5,
    device=DEVICE,
    out_path=tempfile.TemporaryDirectory().name,
)

You can then use its `train()` method.


In [14]:
dense_site_classif.train()

--- Hyperparameters ---
model = DenseSiteNet(
  (dense_layer1): Sequential(
    (0): Linear(in_features=218834, out_features=100, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
  )
  (dense_layer2): Linear(in_features=100, out_features=1, bias=True)
)
split_proportion = 0.9
batch_pad_sequences = False
learning_rate = 0.01
batch_size = 64
max_epochs = 5
early_stopping_patience = 5
--- Creating loaders ---
--- Training start ---
Start training using cpu device.
Epoch   Train loss   Valid loss    F1-score        Lr   Best
------------------------------------------------------------


[1/14]   7%|7          [00:00<?]

    1        0.693        0.693       0.581   0.00000      *


[1/14]   7%|7          [00:00<?]

    2        0.171        0.443       0.771   0.00250      *


...

    3        0.023        0.131       0.967   0.00500      *


[1/14]   7%|7          [00:00<?]

    4        0.004        0.106       0.977   0.00750      *


[1/14]   7%|7          [00:00<?]

    5        0.010        0.188       0.926   0.01000       
--- Training ended ---
Training time: 0:00:17
Best epoch: 4
Best valid loss: 0.106
Best valid accuracy (macro): 0.979
Best F1 score: 0.977


### AA CNN on site composition

This classifier uses a convolutional neural network (CNN) applied on the per-site composition data.

There are two different classifiers depending on wether it is applied on DNA (`DNACnnClassifier`) or AA (`AACnnClassifier`) data. In our case we create an instance of `AACnnClassifier` with its base data, device and `out_path`.

Here we also use the `max_width` argument to remove alignments with sequences with more than 250 sites.


In [15]:
cnn_classif = AACnnClassifier(
    data,
    split_proportion=0.9,
    batch_size=64,
    max_epochs=5,
    device=DEVICE,
    max_width=250,
    out_path=tempfile.TemporaryDirectory().name,
)

2024-10-07 09:48:53   Limiting alignments width to 250
2024-10-07 09:48:53   Removing 945/973 alignments.


We can then use the `train()` method to run the classifier.


In [16]:
cnn_classif.train()

--- Hyperparameters ---
model = AAConvNet(
  (conv_layer): Sequential(
    (0): Conv1d(22, 64, kernel_size=(1,), stride=(1,))
    (1): ReLU()
    (2): AvgPool1d(kernel_size=(247,), stride=(247,), padding=(0,))
    (3): Dropout(p=0.2, inplace=False)
  )
  (dense_layer): Linear(in_features=64, out_features=1, bias=True)
)
split_proportion = 0.9
batch_pad_sequences = False
learning_rate = 0.01
batch_size = 64
max_epochs = 5
early_stopping_patience = 5
--- Creating loaders ---
--- Training start ---
Start training using cpu device.
Epoch   Train loss   Valid loss    F1-score        Lr   Best
------------------------------------------------------------


[1/1] 100%|########## [00:00<?]

    1        0.701        0.631       0.000   0.00000      *


[1/1] 100%|########## [00:00<?]

    2        0.700        0.637       0.000   0.00250       


...

    3        0.698        0.648       0.000   0.00500       


[1/1] 100%|########## [00:00<?]

    4        0.695        0.665       0.000   0.00750       


[1/1] 100%|########## [00:00<?]

    5        0.692        0.685       0.000   0.01000       
--- Training ended ---
Training time: 0:00:00
Best epoch: 1
Best valid loss: 0.631
Best valid accuracy (macro): 1.000
Best F1 score: 0.000


### Attention classifier

**WARNING:** this classifier is not operational yet.

This classifier uses an attention based neural network with a transformer-like architecture taken from the Phyloformer model. It is the only classifier that works on raw sequence data instead of per-MSA or per-site compositions.

To use this classifier you have to create an instance of `AttentionClassifier` with its base data, device and `out_path`. Here we add other additional arguments, notably `max_width` which removes alignments with sequences longer than 500 sites, and `max_height` which removes alignments with more than 20 sequences.


In [17]:
attention_classif = AttentionClassifier(
    data,
    max_width=500,
    max_height=20,
    split_proportion=0.9,
    batch_size=16,
    max_epochs=3,
    device=DEVICE,
    out_path=tempfile.TemporaryDirectory().name,
)

2024-10-07 09:49:04   Limiting alignments width to 500
2024-10-07 09:49:04   Removing 0/28 alignments.
2024-10-07 09:49:04   Limiting alignments height to 20
2024-10-07 09:49:04   Removing 2/28 alignments.


You can then run the classifier with its `train()` method.


In [18]:
attention_classif.train()

--- Hyperparameters ---
model = AttentionNet(
  (rowAttentions): ModuleList(
    (0-3): 4 x FixedKernelMultiAttention(
      (k_proj): Linear(in_features=32, out_features=32, bias=True)
      (q_proj): Linear(in_features=32, out_features=32, bias=True)
      (v_proj): Linear(in_features=32, out_features=32, bias=True)
      (elu): ELU(alpha=1.0)
      (out_proj): Linear(in_features=32, out_features=32, bias=True)
      (atten_drop): Dropout(p=0.0, inplace=False)
      (proj_drop): Dropout(p=0.0, inplace=False)
    )
  )
  (columnAttentions): ModuleList(
    (0-3): 4 x FixedKernelMultiAttention(
      (k_proj): Linear(in_features=32, out_features=32, bias=True)
      (q_proj): Linear(in_features=32, out_features=32, bias=True)
      (v_proj): Linear(in_features=32, out_features=32, bias=True)
      (elu): ELU(alpha=1.0)
      (out_proj): Linear(in_features=32, out_features=32, bias=True)
      (atten_drop): Dropout(p=0.0, inplace=False)
      (proj_drop): Dropout(p=0.0, inplace=False)
 

[1/6]  17%|#6         [00:00<?]

    1        0.757        0.800       0.000   0.00000      *


[1/6]  17%|#6         [00:00<?]

    2        0.707        0.690       0.000   0.00025      *


[1/6]  17%|#6         [00:00<?]

    3        0.712        0.695       0.000   0.00050       
--- Training ended ---
Training time: 0:01:10
Best epoch: 2
Best valid loss: 0.690
Best valid accuracy (macro): 0.444
Best F1 score: 0.000


## Pipeline in Python code or notebook

Another way to use this package is to create a pipeline, which allows to run several classifiers at once on the same data.

To do this in a Python script or notebook, you must create an instance of the `Pipeline` class by giving it the path of empirical data, the path of simulated data, the tokenizer to be used and the output path for the results.


In [18]:
pipeline = Pipeline(
    real_path="../data/sample_lg_s256_gc/empirical/",
    sim_path="../data/sample_lg_s256_gc/simulated/",
    tokenizer=AA_TOKENIZER,
    out_path=tempfile.TemporaryDirectory().name,
)

2024-10-07 09:49:16   Tokenizing real alignments...
2024-10-07 09:49:17   Removing sequences with ambiguous tokens...
2024-10-07 09:49:17   127/8988 sequences have been removed from 84/488 alignements due to ambiguous sites.
2024-10-07 09:49:17   Tokenizing simulated alignments...
2024-10-07 09:49:20   Removing sequences with ambiguous tokens...
2024-10-07 09:49:20   0/8713 sequences have been removed from 0/485 alignements due to ambiguous sites.
2024-10-07 09:49:20   Checking alignments...
2024-10-07 09:49:20   485 renamed keys in simulated aligns.
2024-10-07 09:49:20   Creating labels...
2024-10-07 09:49:20   Merging aligns and labels...
2024-10-07 09:49:20   Total number of alignments: 973


By calling the `preprocess_base_data()` method, the empirical and simulated data will be imported, tokenized and preprocessed.


In [19]:
pipeline.preprocess_data()

2024-10-07 09:49:54   --- Preprocessing base data ---
2024-10-07 09:49:54   Tokenizing real alignments...
2024-10-07 09:49:56   Removing sequences with ambiguous tokens...
2024-10-07 09:49:56   127/8988 sequences have been removed from 84/488 alignements due to ambiguous sites.
2024-10-07 09:49:56   Tokenizing simulated alignments...
2024-10-07 09:49:58   Removing sequences with ambiguous tokens...
2024-10-07 09:49:58   0/8713 sequences have been removed from 0/485 alignements due to ambiguous sites.
2024-10-07 09:49:58   Checking alignments...
2024-10-07 09:49:58   485 renamed keys in simulated aligns.
2024-10-07 09:49:58   Creating labels...
2024-10-07 09:49:58   Merging aligns and labels...
2024-10-07 09:49:58   Total number of alignments: 973


You can then add a classifier to the pipeline by calling the `add_classifier()` method and passing it the classifier name as a string.


In [20]:
pipeline.add_classifier("LogisticRegressionClassifier")

You can pass additional arguments to the classifier by adding them to `add_classifier()`.


In [21]:
pipeline.add_classifier("DenseMsaClassifier", max_epochs=5)

One pipeline specific argument is `out_dir`, which indicates the directory name to use to store the classifier results in `out_path`. By default `out_dir` has the same value as the classifier name, so the results of a `LogisticRegressionClassifier` will be saved into `{out_path}/LogisticRegressionClassifier`. Specifying an `out_dir` allows to run the same classifiers several times with different set of parameters in the same pipeline.


In [22]:
pipeline.add_classifier(
    "LogisticRegressionClassifier", scale_features=False, out_dir="UnscaledLogisticRegression"
)

When wanted classifiers have been added, you can run the pipeline with its `run()` method. All classifiers will be run, with the training log displayed in the standard output, and logs and results saved in different directories.


In [23]:
pipeline.run()

2024-10-07 09:50:17   --- Start running LogisticRegressionClassifier ---
2024-10-07 09:50:17   --- Preprocessing data ---


2024-10-07 09:50:17   --- Start training ---
2024-10-07 09:50:17   Number of cross validation folds: 5
2024-10-07 09:50:17   Shuffle data: False
2024-10-07 09:50:17   Scale features: True
2024-10-07 09:50:17   --- Training ended ---
2024-10-07 09:50:17   Training time: 0:00:00
2024-10-07 09:50:17   Fold F1 scores: ('0.9596', '0.9949', '0.9789', '0.9898', '0.9898')
2024-10-07 09:50:17   --- End running LogisticRegressionClassifier ---
2024-10-07 09:50:17   --- Start running DenseMsaClassifier ---
2024-10-07 09:50:17   --- Hyperparameters ---
2024-10-07 09:50:17   model = DenseMsaNet(
2024-10-07 09:50:17     (dense_layer1): Sequential(
2024-10-07 09:50:17       (0): Linear(in_features=22, out_features=100, bias=True)
2024-10-07 09:50:17       (1): ReLU()
2024-10-07 09:50:17       (2): Dropout(p=0.2, inplace=False)
2024-10-07 09:50:17     )
2024-10-07 09:50:17     (dense_layer2): Linear(in_features=100, out_features=1, bias=True)
2024-10-07 09:50:17   )
2024-10-07 09:50:17   split_proport

## Pipeline from the command line

The third way to use this package is to create and run a pipeline directly from the command line. In this case, the pipeline configuration must be created as a JSON file.

An example file can be found in `config/sample_config.json`:

```json
{
    "real_path": "data/sample_lg_s256_gc/empirical/",
    "sim_path": "data/sample_lg_s256_gc/simulated/",
    "tokenizer": "AA_TOKENIZER",
    "out_path": "runs/test_pipeline",
    "classifiers": [
        {
            "classifier": "LogisticRegressionClassifier",
            "args": { "scale_features": true }
        },
        {
            "classifier": "LogisticRegressionClassifier",
            "args": { "scale_features": false },
            "out_dir": "LogisticRegressionUnscaled"
        },
        {
            "classifier": "DenseMsaClassifier",
            "args": { "batch_size": 32, "max_epochs": 500 }
        },
        { "classifier": "DenseSiteClassifier" },
        {
            "classifier": "AACnnClassifier",
            "args": { "batch_size": 64, "max_epochs": 500 }
        },
        {
            "classifier": "AttentionClassifier",
            "args": { "batch_size": 16, "max_width": 400 }
        }
    ]
}
```

The top level elements of this file are:

-   `real_path`: path to folder with empirical Fasta files
-   `sim_path`: path to folder with simulated Fasta files
-   `tokenizer`: tokenizer to be used, can be `"AA_TOKENIZER"` or `"DNA_TOKENIZER"`
-   `out_path`: path to store output of the pipeline
-   `classifiers`: array containing a list of classifiers to be run on the data

Classifiers in the `classifiers` array are defined with a dictionary with the following entries:

-   `classifier`: the name of the classifier to run (`"DenseMsaClassifier"`, `"AACnnClassifier"`, etc.)
-   `args` (optional): arguments to pass to the classifier init method (depending on the classifier, can be `scale_features`, `batch_size`, `max_width`, etc.)
-   `out_dir` (optional): the directory name to store this classifier output in `out_path`. By default the directory name is the same as the classifier name.

Once the configuration file has been created, you can run the pipeline with:

```shell
uv run python src/classifiers/pipeline.py --config config/sample_config.json --no-progress
# or, equivalently
uv run -m classifiers.pipeline --config config/sample_config.json --no-progress
```
